In [0]:
%pip install mlflow[databricks]>=3.9.0 --upgrade --force-reinstall

In [0]:
import os
import mlflow
from mlflow.exceptions import MlflowException
from mlflow.entities import UCSchemaLocation
from mlflow.tracing.enablement import set_experiment_trace_location

mlflow.set_tracking_uri("databricks")

# Specify the ID of a SQL warehouse you have access to.
os.environ["MLFLOW_TRACING_SQL_WAREHOUSE_ID"] = "aeb85ab1bb2a92e7"
# Specify the name of the MLflow Experiment to use for viewing traces in the UI.
experiment_name = "/Users/hubert.dudek@brickschool.io/trace_testing"
# Specify the name of the Catalog to use for storing traces.
catalog_name = "main"
# Specify the name of the Schema to use for storing traces.
schema_name = "default"

if experiment := mlflow.get_experiment_by_name(experiment_name):
    experiment_id = experiment.experiment_id
else:
    experiment_id = mlflow.create_experiment(name=experiment_name)
print(f"Experiment ID: {experiment_id}")

# To link an experiment to a trace location
result = set_experiment_trace_location(
    location=UCSchemaLocation(catalog_name=catalog_name, schema_name=schema_name),
    experiment_id=experiment_id,
)
print(result.full_otel_spans_table_name)

In [0]:
import os
os.environ["MLFLOW_TRACING_DESTINATION"] = f"main.default"

In [0]:
from mlflow.entities import UCSchemaLocation
mlflow.tracing.set_destination(
    destination=UCSchemaLocation(
        catalog_name=catalog_name,
        schema_name=schema_name,
    )
)

@mlflow.trace
def test(x):
    return x + 1

test(100)